### Data Description 

* emp_length_int : Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
* home_ownership : The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER.
* income_category : Categorized Income (Low, Medium, High) 
* annual_inc : The self-reported annual income provided by the borrower during registration.

* loan_amount : The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
* term : The number of payments on the loan. Values are in months and can be either 36 or 60.

* application_type : Indicates whether the loan is an individual application or a joint application with two co-borrowers
* purpose : A category provided by the borrower for the loan request.
* interest_payments : ;;
* loan_condition : Condition of the Loan [TARGET] (Good Loan = 0 , Bad Loan = 1)
* interest_rate :  Interest Rate on the loan
* grade : LC assigned loan grade
* dti : A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, - - - excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
* total_pymnt : Payments received to date for total amount funded
* total_rec_prncp : Principal received to date
* recoveries : post charge off gross recovery
* installment : The monthly payment owed by the borrower if the loan originates.
* region : region of Loan being executed

### Data Description (Korean)
* year : 대출 발생 연도
* issue_d : 대출 발생 일자
* final_d : 마지막 거래일자
* emp_length_int : "근속년수. 0은 1년 미만, 10은 10년 이상"
* home_ownership : "등록 시 대출자에게서 제공된 집 보유 상태. RENT(대여) = 1, OWN(소유) = 2, MORTAGE(담보대출) = 3 "
* home_ownership_cat : ;;
* income_category : "수익 Low = 1, Medium = 2, High = 3 으로 분류"
* annual_inc : 등록시 대출자에게서 제공된 연간 소득
* income_cat : ;;
* loan_amount : 대출금액(달러)
* term : "대출기간(36개월 = 1, 60개월 = 2)"
* term_cat : ;;
* application_type : "개인 대출 신청(=1) 인지, 2명의 대출자에 의해 공동으로 신청된 대출 신청 (=2)인지 여부"
* application_type_cat : ;;
* purpose : 대출이유
* purpose_cat : "대출용도(빛 청산, 카드 대금 결제, 집 개발 등등) 
[credit_card = 1, car = 2, small_business = 3, other = 4, wedding = 5, debt_consolidation =6, 
home_improvement = 7, major_purchase = 8, medical = 9, moving = 10, vacation = 11, house =12,
renewable_energy = 13,  educational = 14]"
* interest_payments : "이자 지불? (Low = 1, High =2 로 분류)"
* interest_payments_cat : ;;
* loan_condition : 대출의 상태(TARGET) (Good Loan = 0 , Bad Loan = 1)
* loan_condition_cat : ;;
* interest_rate : 대출의 이자율
* grade : 대출 등급 ( A ~ G , 1~7)
* grade_cat : ;;
* dti : 금융부채 상환능력을 소득으로 따져서 대출한도를 정하는 계산비율
* total_pymnt : 총 상환금액
* total_rec_prncp : ???
* recoveries : 회수
* installment : 분할 불입금
* region : 거래지역

### Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score , average_precision_score 
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve ,auc , log_loss ,  classification_report 
from sklearn.preprocessing import StandardScaler , Binarizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
import time
import os, sys, gc, warnings, random, datetime
import math
import shap
import joblib
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.model_selection import StratifiedKFold , cross_val_score
from sklearn.metrics import roc_auc_score

### Import Data

fork from previous EDA kernel https://www.kaggle.com/possiblemanjr/handling-imbalanced-data-eda-small-fe

In [ ]:
df = pd.read_pickle("../input/handling-imbalanced-data-eda-small-fe/df_for_use.pkl")
df_fe = pd.read_pickle("../input/handling-imbalanced-data-eda-small-fe/df_fe.pkl")


### Utilities

In [ ]:
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test, pred)
    print('Confusion Matrix')
    print(confusion)
    print('Auccuracy : {0:.4f}, Precision : {1:.4f} , Recall : {2:.4f} , F1_Score : {3:.4f}'.format(accuracy , precision, recall, f1))
    print('------------------------------------------------------------------------------')

In [ ]:
thresholds = {0.1,0.15, 0.2,0.25, 0.3,0.35, 0.4 , 0.45 , 0.5}

def get_eval_by_threshold(y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold = custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('threshold:', custom_threshold)
        get_clf_eval(y_test, custom_predict)

## get_eval_by_threshold(y_test, pred_proba[:,1].reshape(-1,1), thresholds)

### train_test_split (Stratify)

In [ ]:
X = df.drop('loan_condition_cat', axis=1)
y = df['loan_condition_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 2020, stratify = y)



NGboost

In [ ]:
# !pip install --upgrade git+https://github.com/stanfordmlgroup/ngboost.git

In [ ]:
# from ngboost import NGBRegressor, NGBClassifier
# from ngboost.ngboost import NGBoost
# from ngboost.learners import default_tree_learner
# from ngboost.scores import CRPS, MLE , LogScore
# from ngboost.distns import LogNormal, Normal
# from ngboost.distns import k_categorical, Bernoulli

In [ ]:
# start = time.time()

# ngb_clf = NGBClassifier(Dist=k_categorical(2), verbose=10 ,Score=LogScore, random_state = 2020 )
# ngb_clf.fit(X_train, y_train , early_stopping_rounds=100)


# ngb_runtime = time.time() - start



# # test roc_auc_score
# get_eval_by_threshold(y_test, ngb_clf.predict_proba(X_test)[:,1].reshape(-1,1), thresholds)
# NGb_roc_score = roc_auc_score(y_test, ngb_clf.predict_proba(X_test)[:,1], average = 'macro')



# print( 'NGboost_ROC_AUC : {0:.4f} , Runtime : {1:.4f}'.format(NGb_roc_score ,ngb_runtime ))



In [ ]:
##RandomForest with stratified 5 Fold

In [ ]:
n_estimators = 10
max_features = 'auto'
max_depth = None
min_samples_split = 2
min_samples_leaf = 1
min_weight_fraction_leaf = 0.0
max_leaf_nodes = None
bootstrap = True
oob_score = False
n_jobs = -1
random_state = 2018
class_weight = 'balanced'

RFC = RandomForestClassifier(n_estimators=n_estimators, 
        max_features=max_features, max_depth=max_depth,
        min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
        min_weight_fraction_leaf=min_weight_fraction_leaf, 
        max_leaf_nodes=max_leaf_nodes, bootstrap=bootstrap, 
        oob_score=oob_score, n_jobs=n_jobs, random_state=random_state, 
        class_weight=class_weight)

In [ ]:
##RandomForest with stratified 5 Fold

trainingScores = []
cvScores = []
predictionsBasedOnKFolds = pd.DataFrame(data=[],
                                        index=y_train.index,columns=[0,1])


clf = RandomForestClassifier(n_estimators=30, min_samples_leaf=20, max_features=0.7, n_jobs=-1, random_state = 2020, oob_score=True)
cv = StratifiedKFold(n_splits=5,random_state = 2020)
y_preds_rf = np.zeros(X_test.shape[0])
n_iter = 0 
for train_index,test_index in cv.split(X_train,y_train):
    trx , tsx = X_train.iloc[train_index] , X_train.iloc[test_index]
    vly , vlt = y_train.iloc[train_index] , y_train.iloc[test_index]
    RFC = RFC.fit(trx,vly)   
    loglossTraining = log_loss(vly, \
                                RFC.predict_proba(trx)[:,1])
    trainingScores.append(loglossTraining)
    
    predictionsBasedOnKFolds.loc[tsx.index,:] = \
        RFC.predict_proba(tsx)[:,1]  
    loglossCV = log_loss(vlt, \
        predictionsBasedOnKFolds.loc[tsx.index,1])
    cvScores.append(loglossCV)
    print('Training Log Loss: ', loglossTraining)
    print('CV Log Loss: ', loglossCV)
    
    n_iter += 1
    cv_roc_score = roc_auc_score(y_test, RFC.predict_proba(X_test)[:,1], average = 'macro')
    cv_precision, cv_recall, _ = precision_recall_curve(y_test,RFC.predict_proba(X_test)[:,1])
    cv_pr_auc = auc(cv_recall, cv_precision)
    print( '\n#{0}, CV_ROC_AUC : {1} , RF_CV_PR_AUC : {2} '.format(n_iter ,cv_roc_score, cv_pr_auc))
    y_preds_rf += RFC.predict_proba(X_test)[:,1]/ cv.n_splits

rf_cv_roc_score = roc_auc_score(y_test, y_preds_rf, average = 'macro')
rf_cv_precision, rf_cv_recall, _ = precision_recall_curve(y_test,y_preds_rf)
rf_cv_pr_auc = auc(rf_cv_recall, rf_cv_precision)    
loglossRandomForestsClassifier = log_loss(y_train, 
                                          predictionsBasedOnKFolds.loc[:,1])
print('Random Forests Log Loss: ', loglossRandomForestsClassifier)
    
print( 'RF_cv_ROC_AUC : {0:.4f} , RF_cv_PR_AUC : {1:.4f} '.format(rf_cv_roc_score ,rf_cv_pr_auc ))

In [ ]:
preds = pd.concat([y_train,predictionsBasedOnKFolds.loc[:,1]], axis=1)
preds.columns = ['trueLabel','prediction']
predictionsBasedOnKFoldsRandomForests = preds.copy()

precision, recall, thresholds = precision_recall_curve(preds['trueLabel'],
                                                       preds['prediction'])
average_precision = average_precision_score(preds['trueLabel'],
                                            preds['prediction'])

plt.step(recall, precision, color='k', alpha=0.7, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
          average_precision))

fpr, tpr, thresholds = roc_curve(preds['trueLabel'],preds['prediction'])
areaUnderROC = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic: \
          Area under the curve = {0:0.2f}'.format(
          areaUnderROC))
plt.legend(loc="lower right")
plt.show()

In [ ]:
params_xGB = {
    'nthread':16, # 코어 수
    'gamma': 0, # 감마 : 범위 (0 ~ 무한대) , 디폴트 0
        # 이 값이 높으면 복잡성이 감소(편향 증가, 변동 감소) 
    'max_depth': 6, # max_depth : 범위 (1 ~ 무한대) , 디폴트 6 ## 트리의 최대 깊이
    'min_child_weight': 1, # min_child_weight : 범위 (0 ~ 무한대) , 디폴트 1 ## 자식노드에 필요한 가중치의 최소 합계
    'max_delta_step': 0, # max_delta_step : 범위 (0 ~ 무한대) ,  디폴트 0 ## 각 트리의 가중치 추정을 위한 최대 델타 단계
    'subsample': 1.0, # subsample : 범위 (0 ~ 1) , 디폴트 1
        # 훈련 데이터의 샘플링 비율
    'colsample_bytree': 1.0, # colsample_bytree : 범위 (0 ~ 1) , 디폴트 1
        # 훈련 피쳐의 샘플링 비율
    'objective':'binary:logistic',
    'num_class':1,
    'eval_metric':'logloss',
    'seed':2020,
    'tree_method' : 'gpu_hist',
}

In [ ]:
trainingScores = []
cvScores = []
predictionsBasedOnKFolds = pd.DataFrame(data=[],
                                    index=y_train.index,columns=['prediction'])
k_fold = StratifiedKFold(n_splits=5,shuffle=True,random_state=2020)

for train_index, cv_index in k_fold.split(np.zeros(len(X_train)),
                                          y_train.ravel()):
    X_train_fold, X_cv_fold = X_train.iloc[train_index,:], \
        X_train.iloc[cv_index,:]
    y_train_fold, y_cv_fold = y_train.iloc[train_index], \
        y_train.iloc[cv_index]
    
    dtrain = xgb.DMatrix(data=X_train_fold, label=y_train_fold)
    dCV = xgb.DMatrix(data=X_cv_fold)
    
    bst = xgb.cv(params_xGB, dtrain, num_boost_round=2000, 
                 nfold=5, early_stopping_rounds=200, verbose_eval=50)
    
    # 수정 사항 : np.arrary 로 재정의 하면서 경고 메세지를 지울 수 있음
    best_rounds = np.argmin(np.array(bst['test-logloss-mean']))
    bst = xgb.train(params_xGB, dtrain, best_rounds)
    
    loglossTraining = log_loss(y_train_fold, bst.predict(dtrain))
    trainingScores.append(loglossTraining)
    
    predictionsBasedOnKFolds.loc[X_cv_fold.index,'prediction'] = \
        bst.predict(dCV)
    loglossCV = log_loss(y_cv_fold, \
        predictionsBasedOnKFolds.loc[X_cv_fold.index,'prediction'])
    cvScores.append(loglossCV)
    
    print('Training Log Loss: ', loglossTraining)
    print('CV Log Loss: ', loglossCV)
    
loglossXGBoostGradientBoosting = \
    log_loss(y_train, predictionsBasedOnKFolds.loc[:,'prediction'])
print('XGBoost Gradient Boosting Log Loss: ', loglossXGBoostGradientBoosting)

In [ ]:
preds = pd.concat([y_train,predictionsBasedOnKFolds.loc[:,'prediction']], axis=1)
preds.columns = ['trueLabel','prediction']
predictionsBasedOnKFoldsXGBoostGradientBoosting = preds.copy()

precision, recall, thresholds = \
    precision_recall_curve(preds['trueLabel'],preds['prediction'])
average_precision = \
    average_precision_score(preds['trueLabel'],preds['prediction'])

plt.step(recall, precision, color='k', alpha=0.7, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
          average_precision))

fpr, tpr, thresholds = roc_curve(preds['trueLabel'],preds['prediction'])
areaUnderROC = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic: \
        Area under the curve = {0:0.2f}'.format(areaUnderROC))
plt.legend(loc="lower right")
plt.show()

In [ ]:
params_lightGB = {
    'task': 'train',
    'application':'binary',
    'num_class':1,
    'boosting': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'metric_freq':50,
    'is_training_metric':False,
    'max_depth':4,
    'num_leaves': 31,
#     'learning_rate': 0.01,
    'feature_fraction': 1.0,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'bagging_seed': 2020,
    'verbose': 50,
    'num_threads':16,
    'random_state ' : 2020
}



In [ ]:
trainingScores = []
cvScores = []
predictionsBasedOnKFolds = pd.DataFrame(data=[],
                                index=y_train.index,columns=['prediction'])

for train_index, cv_index in k_fold.split(np.zeros(len(X_train)),
                                          y_train.ravel()):
    X_train_fold, X_cv_fold = X_train.iloc[train_index,:], \
        X_train.iloc[cv_index,:]
    y_train_fold, y_cv_fold = y_train.iloc[train_index], \
        y_train.iloc[cv_index]
    
    lgb_train = lgb.Dataset(X_train_fold, y_train_fold)
    lgb_eval = lgb.Dataset(X_cv_fold, y_cv_fold, reference=lgb_train)
    gbm = lgb.train(params_lightGB, lgb_train, num_boost_round=10000,
                   valid_sets=lgb_eval, early_stopping_rounds=200 , verbose_eval = 500)
    
    loglossTraining = log_loss(y_train_fold, \
                gbm.predict(X_train_fold, num_iteration=gbm.best_iteration))
    trainingScores.append(loglossTraining)
    
    predictionsBasedOnKFolds.loc[X_cv_fold.index,'prediction'] = \
        gbm.predict(X_cv_fold, num_iteration=gbm.best_iteration) 
    loglossCV = log_loss(y_cv_fold, \
        predictionsBasedOnKFolds.loc[X_cv_fold.index,'prediction'])
    cvScores.append(loglossCV)
    
    print('Training Log Loss: ', loglossTraining)
    print('CV Log Loss: ', loglossCV)
    
loglossLightGBMGradientBoosting = \
    log_loss(y_train, predictionsBasedOnKFolds.loc[:,'prediction'])
print('LightGBM Gradient Boosting Log Loss: ', loglossLightGBMGradientBoosting)

In [ ]:
preds = pd.concat([y_train,predictionsBasedOnKFolds.loc[:,'prediction']], axis=1)
preds.columns = ['trueLabel','prediction']
predictionsBasedOnKFoldsLightGBMGradientBoosting = preds.copy()

precision, recall, thresholds = \
    precision_recall_curve(preds['trueLabel'],preds['prediction'])
average_precision = \
    average_precision_score(preds['trueLabel'],preds['prediction'])

plt.step(recall, precision, color='k', alpha=0.7, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
          average_precision))

fpr, tpr, thresholds = roc_curve(preds['trueLabel'],preds['prediction'])
areaUnderROC = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic: \
Area under the curve = {0:0.2f}'.format(areaUnderROC))
plt.legend(loc="lower right")
plt.show()

In [ ]:
predictionsTestSetRandomForests = \
    pd.DataFrame(data=[],index=y_test.index,columns=['prediction'])
predictionsTestSetRandomForests.loc[:,'prediction'] = \
    RFC.predict_proba(X_test)[:,1]
logLossTestSetRandomForests = \
    log_loss(y_test, predictionsTestSetRandomForests)

In [ ]:
predictionsTestSetXGBoostGradientBoosting = \
    pd.DataFrame(data=[],index=y_test.index,columns=['prediction'])
dtest = xgb.DMatrix(data=X_test)
predictionsTestSetXGBoostGradientBoosting.loc[:,'prediction'] = \
    bst.predict(dtest)
logLossTestSetXGBoostGradientBoosting = \
    log_loss(y_test, predictionsTestSetXGBoostGradientBoosting)

In [ ]:
predictionsTestSetLightGBMGradientBoosting = \
    pd.DataFrame(data=[],index=y_test.index,columns=['prediction'])
predictionsTestSetLightGBMGradientBoosting.loc[:,'prediction'] = \
    gbm.predict(X_test, num_iteration=gbm.best_iteration)
logLossTestSetLightGBMGradientBoosting = \
    log_loss(y_test, predictionsTestSetLightGBMGradientBoosting)

In [ ]:

print("Log Loss of Random Forests on Test Set: ", \
          logLossTestSetRandomForests)
print("Log Loss of XGBoost Gradient Boosting on Test Set: ", \
          logLossTestSetXGBoostGradientBoosting)
print("Log Loss of LightGBM Gradient Boosting on Test Set: ", \
          logLossTestSetLightGBMGradientBoosting)

In [ ]:
precision, recall, thresholds = \
    precision_recall_curve(y_test,predictionsTestSetRandomForests)
average_precision = \
    average_precision_score(y_test,predictionsTestSetRandomForests)

plt.step(recall, precision, color='k', alpha=0.7, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
          average_precision))

fpr, tpr, thresholds = roc_curve(y_test,predictionsTestSetRandomForests)
areaUnderROC = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic: \
Area under the curve = {0:0.2f}'.format(areaUnderROC))
plt.legend(loc="lower right")
plt.show()

In [ ]:
precision, recall, thresholds = \
    precision_recall_curve(y_test,predictionsTestSetXGBoostGradientBoosting)
average_precision = \
    average_precision_score(y_test,predictionsTestSetXGBoostGradientBoosting)

plt.step(recall, precision, color='k', alpha=0.7, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
          average_precision))

fpr, tpr, thresholds = \
    roc_curve(y_test,predictionsTestSetXGBoostGradientBoosting)
areaUnderROC = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic: \
Area under the curve = {0:0.2f}'.format(areaUnderROC))
plt.legend(loc="lower right")
plt.show()

In [ ]:
precision, recall, thresholds = \
    precision_recall_curve(y_test,predictionsTestSetLightGBMGradientBoosting)
average_precision = \
    average_precision_score(y_test,predictionsTestSetLightGBMGradientBoosting)

plt.step(recall, precision, color='k', alpha=0.7, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
          average_precision))

fpr, tpr, thresholds = \
    roc_curve(y_test,predictionsTestSetLightGBMGradientBoosting)
areaUnderROC = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic: \
Area under the curve = {0:0.2f}'.format(areaUnderROC))
plt.legend(loc="lower right")
plt.show()

In [ ]:
# 앙상블

In [ ]:
predictionsBasedOnKFoldsFourModels = pd.DataFrame(data=[],index=y_train.index)
predictionsBasedOnKFoldsFourModels = predictionsBasedOnKFoldsFourModels.join(predictionsBasedOnKFoldsRandomForests['prediction'] \
    .astype(float),how='left').join( \
    predictionsBasedOnKFoldsXGBoostGradientBoosting['prediction'].astype(float), \
    how='left',rsuffix="2").join( \
    predictionsBasedOnKFoldsLightGBMGradientBoosting['prediction'].astype(float), \
    how='left',rsuffix="3")
predictionsBasedOnKFoldsFourModels.columns = \
    ['predsRF','predsXGB','predsLightGBM']

In [ ]:
X_trainWithPredictions = \
    X_train.merge(predictionsBasedOnKFoldsFourModels,
                  left_index=True,right_index=True)

In [ ]:
params_lightGB = {
    'task': 'train',
    'application':'binary',
    'num_class':1,
    'boosting': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'metric_freq':50,
    'is_training_metric':False,
    'max_depth':4,
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 1.0,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'bagging_seed': 2018,
    'verbose': 0,
    'num_threads':16
}

In [ ]:
trainingScores = []
cvScores = []
predictionsBasedOnKFoldsEnsemble = \
    pd.DataFrame(data=[],index=y_train.index,columns=['prediction'])

for train_index, cv_index in k_fold.split(np.zeros(len(X_train)), \
                                          y_train.ravel()):
    X_train_fold, X_cv_fold = \
        X_trainWithPredictions.iloc[train_index,:], \
        X_trainWithPredictions.iloc[cv_index,:]
    y_train_fold, y_cv_fold = y_train.iloc[train_index], y_train.iloc[cv_index]
    
    lgb_train = lgb.Dataset(X_train_fold, y_train_fold)
    lgb_eval = lgb.Dataset(X_cv_fold, y_cv_fold, reference=lgb_train)
    gbm = lgb.train(params_lightGB, lgb_train, num_boost_round=10000,
                   valid_sets=lgb_eval, early_stopping_rounds=200 ,verbose_eval = 500)
    
    loglossTraining = log_loss(y_train_fold, \
        gbm.predict(X_train_fold, num_iteration=gbm.best_iteration))
    trainingScores.append(loglossTraining)
    
    predictionsBasedOnKFoldsEnsemble.loc[X_cv_fold.index,'prediction'] = \
        gbm.predict(X_cv_fold, num_iteration=gbm.best_iteration) 
    loglossCV = log_loss(y_cv_fold, \
        predictionsBasedOnKFoldsEnsemble.loc[X_cv_fold.index,'prediction'])
    cvScores.append(loglossCV)
    
    print('Training Log Loss: ', loglossTraining)
    print('CV Log Loss: ', loglossCV)
    
loglossEnsemble = log_loss(y_train, \
        predictionsBasedOnKFoldsEnsemble.loc[:,'prediction'])
print('Ensemble Log Loss: ', loglossEnsemble)

In [ ]:
print('Feature importances:', list(gbm.feature_importance()))

In [ ]:
preds = pd.concat([y_train,predictionsBasedOnKFoldsEnsemble.loc[:,'prediction']], axis=1)
preds.columns = ['trueLabel','prediction']

precision, recall, thresholds = \
    precision_recall_curve(preds['trueLabel'],preds['prediction'])
average_precision = \
    average_precision_score(preds['trueLabel'],preds['prediction'])

plt.step(recall, precision, color='k', alpha=0.7, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
          average_precision))

fpr, tpr, thresholds = roc_curve(preds['trueLabel'],preds['prediction'])
areaUnderROC = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic: \
Area under the curve = {0:0.2f}'.format(areaUnderROC))
plt.legend(loc="lower right")
plt.show()

In [ ]:
scatterData = predictionsTestSetLightGBMGradientBoosting.join(y_test,how='left')
scatterData.columns = ['Predicted Probability','True Label']
ax = sns.regplot(x="True Label", y="Predicted Probability", color='k', 
                 fit_reg=False, scatter_kws={'alpha':0.1},
                 data=scatterData).set_title( \
                'Plot of Prediction Probabilities and the True Label')

In [ ]:
scatterDataMelted = pd.melt(scatterData, "True Label", \
                            var_name="Predicted Probability")
scatterDataMelted.head()
ax = sns.stripplot(x="value", y="Predicted Probability", \
                   hue='True Label', jitter=0.4, \
                   data=scatterDataMelted).set_title( \
                   'Plot of Prediction Probabilities and the True Label')

In [ ]:
## XGboost with Stratified 5 Fold

val = np.zeros(X_train.shape[0])
pred = np.zeros(X_test.shape[0])
folds = StratifiedKFold(n_splits=5, random_state=2020)

model_xgb =  xgb.XGBClassifier(
                              n_estimators=10000,
                              objective='binary:logistic', 

                              verbosity =1,
                              eval_metric  = 'aucpr',
                              tree_method='gpu_hist',
                              random_state = 2020, 
                              n_jobs=-1)

for fold_index, (train_index,val_index) in enumerate(folds.split(X_train,y_train)):
    print('Batch {} started...'.format(fold_index))
    gc.collect()
    bst = model_xgb.fit(X_train.iloc[train_index],y_train.iloc[train_index],
              eval_set = [(X_train.iloc[val_index],y_train.iloc[val_index])],
              early_stopping_rounds=200,
              verbose= 100, 
              eval_metric ='aucpr'
              )

    pred += bst.predict_proba(X_test)[:,1]/folds.n_splits

xgb_cv_roc_score = roc_auc_score(y_test, pred, average = 'macro')
xgb_cv_precision, xgb_cv_recall, _ = precision_recall_curve(y_test,pred)
xgb_cv_pr_auc = auc(xgb_cv_recall, xgb_cv_precision)



print( 'XGboost_cv_ROC_AUC : {0:.4f} , XGboost_cv_PR_AUC : {1:.4f} '.format(xgb_cv_roc_score ,xgb_cv_pr_auc ))


In [ ]:
###LightGBM with Stratified 5 Fold

X = df.drop('loan_condition_cat', axis=1)
y = df['loan_condition_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 , random_state = 2020, stratify = y)


from lightgbm import LGBMClassifier

from time import time
params_lgb={'boosting_type':'gbdt',
           'objective': 'binary',
           'random_state':2020,
           'metric':'binary_logloss',
            'metric_freq' : 50,
            'max_depth' :4, 
            'num_leaves' : 31,
            'learning_rate' : 0.01,
            'feature_fraction' : 1.0,
            'bagging_fraction' : 1.0,
            'bagging_freq' : 0,
            'bagging_seed' : 2020,
            'num_threads' : 16
           }

k_fold=5
kf=StratifiedKFold(n_splits=k_fold,shuffle=True, random_state=2020)
training_start_time = time()
aucs=[]
y_preds = np.zeros(X_test.shape[0])

for fold, (trn_idx,val_idx) in enumerate(kf.split(X_train,y_train)):
    start_time = time()
    print('Training on fold {}'.format(fold + 1))
    trn_data = lgb.Dataset(X_train.iloc[trn_idx], label=y_train.iloc[trn_idx])
    val_data = lgb.Dataset(X_train.iloc[val_idx], label=y_train.iloc[val_idx])
    clf = lgb.train(params_lgb, trn_data, num_boost_round=10000, valid_sets = [trn_data, val_data], 
                    verbose_eval=200, early_stopping_rounds=200)
#     aucs.append(clf.best_score['valid_1']['auc'])
    print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    y_preds += clf.predict(X_test) / 5
    
    
    
print('-' * 30)
print('Training is completed!.')
# print("\n## Mean CV_AUC_Score : ", np.mean(aucs))
print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
# print(clf.best_params_)
print('-' * 30)



lgbm_cv_roc_score = roc_auc_score(y_test, y_preds, average = 'macro')
lgbm_cv_precision, lgbm_cv_recall, _ = precision_recall_curve(y_test,y_preds)
lgbm_cv_pr_auc = auc(lgbm_cv_recall, lgbm_cv_precision)



print( 'LightGBM_cv_ROC_AUC : {0:.4f} , LightGBM_cv_PR_AUC : {1:.4f} '.format(lgbm_cv_roc_score ,lgbm_cv_pr_auc ))



In [ ]:
## Results

print( 'RandomForest_ROC_AUC : {0:.4f} , RandomForest_PR_AUC : {1:.4f} , Runtime : {2:.4f}'.format(rf_roc_score ,rf_pr_auc, RF_runtime ))
print( 'XGboost_gpu_ROC_AUC : {0:.4f} , XGboost_gpu_PR_AUC : {1:.4f} , Runtime : {2:.4f}'.format(xgb_gpu_roc_score ,xgb_gpu_pr_auc, xgb_gpu_runtime ))
print( 'XGboost_cpu_ROC_AUC : {0:.4f} , XGboost_cpu_PR_AUC : {1:.4f} , Runtime : {2:.4f}'.format(xgb_cpu_roc_score ,xgb_cpu_pr_auc, xgb_cpu_runtime ))
print( 'LightGBM_ROC_AUC : {0:.4f} , LightGBM_PR_AUC : {1:.4f} ,Runtime : {2:.4f}'.format(lgbm_roc_score ,lgbm_pr_auc, lgbm_cpu_runtime))
print( 'RF_cv_ROC_AUC : {0:.4f} , RF_cv_PR_AUC : {1:.4f} '.format(rf_cv_roc_score ,rf_cv_pr_auc ))
print( 'XGboost_cv_ROC_AUC : {0:.4f} , XGboost_cv_PR_AUC : {1:.4f} '.format(xgb_cv_roc_score ,xgb_cv_pr_auc ))
print( 'LightGBM_cv_ROC_AUC : {0:.4f} , LightGBM_cv_PR_AUC : {1:.4f} '.format(lgbm_cv_roc_score ,lgbm_cv_pr_auc ))


In [ ]:
########## Export


#save model
joblib.dump(rf_clf , 'rf_clf.pkl')
joblib.dump(lgbm_clf , 'lgbm_clf.pkl')
joblib.dump(xgb_clf , 'xgb_clf.pkl')
# joblib.dump(ngb_clf , 'ngb_clf.pkl')



